In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from compute_flops import estimate_max_training_steps

max_steps = estimate_max_training_steps(
    data_path="../lotka_volterra_data.h5",
    lora_rank=2,
    input_fraction=0.7,
    batch_size=4,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)


/Users/raunaqrai/miniconda_x86_64/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/raunaqrai/miniconda_x86_64/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



🔢 Avg token count: 512
⚙️  FLOPs per training step (batch): 6.75e+12
🧪 Evaluation FLOPs: 1.69e+14
🎯 Remaining budget for training: 9.98e+16
🚀 Max training steps allowed: 14781


In [6]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from compute_flops import compute_flops

compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=0.7,
    lora_rank=8,
    batch_size=4,
    training_steps=1000,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 6.76e+15
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 6.93e+15
 🎯 Percentage of FLOP budget used: 6.92927%


- Start with 1000 time series
- Split immediately → 700 for training, 300 for evaluation
- Tokenise the 700 training series into sequences
- Train the model on this
- Evaluate the model on the validation/test split (the 300)

In [9]:
import pandas as pd

# FLOPs constants (from your setup)
FLOPS_PER_STEP = 6.76e12
EVAL_FLOPS = 1.69e14
FLOP_BUDGET = 1e17

# Define experiment blocks
experiments = [
    {"Part": "a i", "Experiment": "Untrained", "LoRA Rank": 4, "Learning Rate": 1e-5, "Context Length": 512, "Max Steps": 0, "Average Validation Loss":1.1926},
    {"Part": "a ii", "Experiment": "Default LoRA (4, 1e-5)", "LoRA Rank": 4, "Learning Rate": 1e-5, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.8880},
    {"Part": "b i", "Experiment": "LoRA (2, 1e-5)", "LoRA Rank": 2, "Learning Rate": 1e-5, "Context Length": 512, "Max Steps": 0, "Average Validation Loss":0},
    {"Part": "b ii (same as a ii, so dont have to run it)", "Experiment": "LoRA (4, 1e-5)", "LoRA Rank": 4, "Learning Rate": 1e-5, "Context Length": 512, "Max Steps": 0, "Average Validation Loss":0.8880},
    {"Part": "b iii", "Experiment": "LoRA (8, 1e-5)", "LoRA Rank": 8, "Learning Rate": 1e-5, "Context Length": 512, "Max Steps": 0, "Average Validation Loss":0},
    {"Part": "b iv", "Experiment": "LoRA (2, 5e-5)", "LoRA Rank": 2, "Learning Rate": 5e-5, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.7193},
    {"Part": "b v", "Experiment": "LoRA (4, 5e-5)", "LoRA Rank": 4, "Learning Rate": 5e-5, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.6628},
    {"Part": "b vi", "Experiment": "LoRA (8, 5e-5)", "LoRA Rank": 8, "Learning Rate": 5e-5, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.6244},
    {"Part": "b vii", "Experiment": "LoRA (2, 1e-4)", "LoRA Rank": 2, "Learning Rate": 1e-4, "Context Length": 512, "Max Steps": 0, "Average Validation Loss":0},
    {"Part": "b viii", "Experiment": "LoRA (4, 1e-4)", "LoRA Rank": 4, "Learning Rate": 1e-4, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.5934},
    {"Part": "b ix", "Experiment": "LoRA (8, 1e-4)", "LoRA Rank": 8, "Learning Rate": 1e-4, "Context Length": 512, "Max Steps": 1000, "Average Validation Loss":0.5594},
    {"Part": "b x", "Experiment": "Best config (CL=128)", "LoRA Rank": 8, "Learning Rate": 1e-4, "Context Length": 128, "Max Steps": 1000, "Average Validation Loss":0.6771},
    {"Part": "b xi", "Experiment": "Best config (CL=768)", "LoRA Rank": 8, "Learning Rate": 1e-4, "Context Length": 768, "Max Steps": 1000, "Average Validation Loss":0.3744}
]

# Create DataFrame
df = pd.DataFrame(experiments)

# Compute Training FLOPs
df["Training FLOPs"] = df["Max Steps"] * FLOPS_PER_STEP

# Compute Total FLOPs for each experiment (training + one-time evaluation, or 0 if no training)
df["Total FLOPs Used"] = df["Training FLOPs"].apply(lambda x: 0 if x == 0 else x + EVAL_FLOPS)


# Compute cumulative across rows (for budget planning)
df["Cumulative FLOPs"] = df["Total FLOPs Used"].cumsum()

# Flag if budget is exceeded
df["Exceeds Budget"] = df["Cumulative FLOPs"] > FLOP_BUDGET

# Formatting
df["Training FLOPs"] = df["Training FLOPs"].apply(lambda x: f"{x:.2e}")
df["Total FLOPs Used"] = df["Total FLOPs Used"].apply(lambda x: f"{x:.2e}")
df["Cumulative FLOPs"] = df["Cumulative FLOPs"].apply(lambda x: f"{x:.2e}")

# Show the table
df


,Part,Experiment,LoRA Rank,Learning Rate,Context Length,Max Steps,Average Validation Loss,Training FLOPs,Total FLOPs Used,Cumulative FLOPs,Exceeds Budget
0,a i,Untrained,4,0.00001,512,0,1.1926,0.00e+00,0.00e+00,0.00e+00,False
1,a ii,"Default LoRA (4, 1e-5)",4,0.00001,512,1000,0.8880,6.76e+15,6.93e+15,6.93e+15,False
2,b i,"LoRA (2, 1e-5)",2,0.00001,512,0,0.0000,0.00e+00,0.00e+00,6.93e+15,False
3,"b ii (same as a ii, so dont have to run it)","LoRA (4, 1e-5)",4,0.00001,512,0,0.8880,0.00e+00,0.00e+00,6.93e+15,False
4,b iii,"LoRA (8, 1e-5)",8,0.00001,512,0,0.0000,0.00e+00,0.00e+00,6.93e+15,False
5,b iv,"LoRA (2, 5e-5)",2,0.00005,512,1000,0.7193,6.76e+15,6.93e+15,1.39e+16,False
6,b v,"LoRA (4, 5e-5)",4,0.00005,512,1000,0.6628,6.76e+15,6.93e+15,2.08e+16,False
7,b vi,"LoRA (8, 5e-5)",8,0.00005,512,1000,0.6244,6.76e+15,6.93e+15,2.77e+16,False
8,b vii,"LoRA (2, 1e-4)",2,0.00010,512,0,0.0000,0.00e+00,0.00e+00,2.77e+16,False
9,b viii,"LoRA (4, 1e-4)",4,0.00010,512,1000,0.5934,6.76e+15,6.93e+15,3.46e+16,False
